In [11]:
import pandas as pd
import itertools
from sklearn import preprocessing
from sklearn import model_selection
import numpy as np
import os
import math
import datetime
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn import svm

pd.set_option('display.max_columns', 500)

In [12]:
### import the csvs, merge on 'id' column, export to csv
data_test = pd.read_csv('watertabletestingvalues.csv')
data_values = pd.read_csv('watertabletrainingvalues.csv')
data_labels = pd.read_csv('watertabletraininglabels.csv')
data_values.merge(data_labels[['id','status_group']],on='id').to_csv('waterabletrainingset.csv')

In [14]:
data = pd.read_csv('waterabletrainingset.csv').drop(columns=['Unnamed: 0']).dropna().reset_index()
data

,index,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
2,5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional
3,13,50495,0.0,2013-03-15,Lawatefuka Water Supply,1368,Lawatefuka water sup,37.092574,-3.181783,Kwa John Izack Mmari,0,Pangani,Omarini,Kilimanjaro,3,7,Siha,Siha Kaskazini,1,True,GeoData Consultants Ltd,Water Board,BL Bondeni,True,2009,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
4,14,53752,0.0,2012-10-20,Biore,0,WEDECO,34.364073,-3.629333,Mwabasabi,0,Internal,Mwabasabi,Shinyanga,17,6,Meatu,Nkoma,0,True,GeoData Consultants Ltd,WUG,None,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
5,16,48451,500.0,2011-07-04,Unicef,1703,DWE,34.642439,-9.106185,Kwa John Mtenzi,0,Rufiji,Kidudumo,Iringa,11,4,Njombe,Mdandu,35,True,GeoData Consultants Ltd,WUA,wanging'ombe water supply s,True,1978,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,dry,dry,river,river/lake,surface,communal standpipe,communal standpipe,non functional
6,17,58155,0.0,2011-09-04,Unicef,1656,DWE,34.569266,-9.085515,Kwa Rose Chaula,0,Rufiji,Yeriko,Iringa,11,4,Njombe,Usuka,50,True,GeoData Consultants Ltd,WUA,wanging'ombe water supply s,True,1978,gravity,gravity,gravity,wua,user-group,pay when scheme fails,on failure,soft,good,dry,dry,river,river/lake,surface,communal standpipe,communal standpipe,non functional
7,19,18274,500.0,2011-02-22,Danida,1763,Danid,34.508967,-9.894412,none,0,Lake Nyasa,Manyanya,Iringa,11,5,Ludewa,Mawengi,1,True,GeoData Consultants Ltd,VWC,Makanj,True,1992,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8,20,48375,200.0,2011-02-27,Twe,2216,TWE,34.473430,-9.594990,none,0,Lake Nyasa,Majengo,Iringa,11,4,Njombe,Matola,4,True,GeoData Consultants Ltd,VWC,Kidabu,False,2008,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
9,22,58500,0.0,2011-10-04,Unicef,1510,DWE,34.586901,-8.980014,Shuleni,0,Rufiji,Mkanivega,Iringa,11,4,Njombe,Usuka,350,True,GeoData Consultants Ltd,WUA,wanging'ombe water supply s,True,1978,gravity,gravity,gravity,wua,user-group,never pay,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,fu

In [7]:
### IGNORE THIS CELL, 
category = 'status_group'

print(data[category].value_counts())
print(data[category].nunique())

for thing in data_test[category].unique():
    if thing not in data[category].unique():
        print(thing+ ' in test not train')

for thing in data[category].unique():
    if thing not in data_test[category].unique():
        print(thing+ ' in train not test')

functional                 16056
non functional              9744
functional needs repair     2013
Name: status_group, dtype: int64
3


KeyError: 'status_group'

In [15]:
# Calculate well age
data['date_recorded_dt'] = pd.to_datetime(data['date_recorded'],format='%Y-%m-%d')
average_construction_year = data['construction_year'][(data['construction_year'] != 0)].mean()
data['age'] = (data['date_recorded_dt'].dt.year + data['date_recorded_dt'].dt.dayofyear/365) - data['construction_year'].replace(0,average_construction_year)


In [16]:
# In this dataset, null values are also written as 0. 
# function that standardizes the data ignoring the 0 values

def zero_isnan_standardize(data_series):
    
    average_value = data_series[(data_series != 0)].mean()
    X = data_series.replace(0,math.nan).values
    Xz = (X - np.nanmean(X, axis=0))/np.nanstd(X, axis=0)
    Xz[np.isnan(Xz)] = 0
    
    return Xz

In [17]:
topfunders = data['funder'].unique()[:31]
topinstallers = data['installer'].unique()[:31]


In [21]:
def convert_funder(row):
    if row in topfunders:
        return row
    else:
        return 'other'
def convert_installer(row):
    if row in topinstallers:
        return row
    else:
        return 'other'
def statusmap(row):
    if row == 'functional':
        return 0
    elif row == 'non functional':
        return 2
    else:
        return 1
data['status_group_int'] = data['status_group'].apply(statusmap)
data['converted_funder'] = data['funder'].apply(convert_funder)
data['converted_installer'] = data['installer'].apply(convert_installer)
data['status_group_int']

0        0
1        0
2        0
3        0
4        0
5        2
6        2
7        0
8        0
9        1
10       0
11       0
12       1
13       0
14       2
15       0
16       0
17       1
18       0
19       2
20       0
21       0
22       0
23       0
24       0
25       1
26       2
27       0
28       0
29       0
        ..
27783    1
27784    2
27785    2
27786    0
27787    2
27788    2
27789    0
27790    2
27791    1
27792    0
27793    0
27794    2
27795    2
27796    2
27797    0
27798    0
27799    0
27800    0
27801    1
27802    0
27803    0
27804    0
27805    0
27806    0
27807    2
27808    2
27809    2
27810    2
27811    0
27812    0
Name: status_group_int, Length: 27813, dtype: int64

In [22]:
### Function to binarize data
def binarize_strings(data_series):
    encoder = preprocessing.LabelBinarizer(sparse_output=False)
    status_array = encoder.fit_transform(data_series)
    
    return pd.DataFrame(columns=encoder.classes_,data=status_array)

In [69]:
# boolean mapping for data with NaN values
def map_bool(data_series):
    mapping = {
        True: 1,
        #replace NaN with average
        math.nan: data_series[(data_series != math.nan)].mean(),
        False: 0
    }
    return data_series.map(mapping).values

In [70]:
### Standardize Numerical Data
#list of columns to standardize with 0 as null values
standardize_columns = ['amount_tsh', # Total static head
                       'gps_height', # Altitude of the well
                       'longitude','latitude', # For approximate locations
                       'population', # Population around the well
                      ]

scaler = preprocessing.StandardScaler()

#new dataframe
data_standard = pd.DataFrame()

for column in standardize_columns:
    
    data_standard[column+'_standard'] = zero_isnan_standardize(data[column])

# standardize regular data
data_standard['age_standard'] = scaler.fit_transform(data['age'].values.reshape(-1,1))

In [71]:
bool_columns = ['permit', #If the waterpoint is permitted
                'public_meeting'
               ]
data_bool = pd.DataFrame()
for column in bool_columns:
    data_bool[column+'_mapped'] = map_bool(data[column])

In [72]:
### Binarize Text Data
binarize_columns = [
                    'lga',
                    'extraction_type', #experiment with extraction_type_group or extraction_type_class
                    'management_group', #who manages well
                    'quality_group', #quality of water, experiment with water_quality
                    'payment_type', #how users pay
                    'quantity', #experiment with quantity_group
                    'source', #experiment with source or source_class
                    'waterpoint_type', #experiment with waterpoint type group
                    'water_quality',
                    'basin',
                    'region',
                    'converted_installer',
                    'converted_funder'
                   ]
data_binarize = pd.DataFrame()
for column in binarize_columns:
    data_binarize = pd.concat([data_binarize,binarize_strings(data[column])],axis=1)

In [73]:
### Final Processed DataFrame
data_processed = pd.concat([data[['id','status_group']],
                            data_standard,
                            data_bool,
                            data_binarize
                           ],axis=1)
data_processed.to_csv('waterabletrainingsetprocessed.csv')


In [74]:
data_cluster = data_processed[data_processed.columns.difference(['id','status_group','index'])]
data_processed.shape[0]

27813

In [75]:
def assign_samples(data,sections):
    #Take data and randomly assign them groups 1-10
    
    points_per_section = int(data.shape[0]/sections)
    
    sampled_data = pd.DataFrame()
    for i in range(sections):

        current_sample = data.sample(n=points_per_section)
        
        data = data.drop(current_sample.index)
        current_sample['random_group'] = i
        sampled_data = pd.concat([sampled_data,current_sample])
        
        
    return sampled_data.sort_index().reset_index(drop=True)

In [76]:
def build_model(training_data):
    #uses training data to train model
    
    training_classes = training_data['status_group'].values
    training_inputs = training_data[training_data.columns.difference(['id','status_group','index'])].values
    model = linear_model.LogisticRegression()
    model.fit(training_inputs, training_classes)
    
    return model

In [77]:
def sample_proba(data,section):
    #given our data (sampled) and specifying a section, 
    # build a model with training data and then find the probabilities for the test data
    
    training_data = data[data['random_group'] != section]
    testing_data = data[data['random_group'] == section]
    
    model = build_model(training_data)
    
    return test_proba(testing_data,model)


In [78]:
def test_proba(testing_data,model):
    #test probabilities
    
    testing_inputs = testing_data[testing_data.columns.difference(['id','status_group','index'])].values
    testing_data['prediction'] = model.predict(testing_inputs)
#     testing_data['prediction0'] = list(list(zip(*model.predict(testing_inputs)))[0])
#     testing_data['prediction1'] = list(list(zip(*model.predict_proba(testing_inputs)))[1])
#     testing_data['prediction2'] = list(list(zip(*model.predict_proba(testing_inputs)))[2])
    
    return testing_data

In [79]:
def run_x_fold(sample_data,sections):
    #find area under curve for each section and average
    testing_data = sample_proba(sample_data,0)
    
    testing_data['validate'] = np.where(testing_data['status_group']==testing_data['prediction'], 1, 0)
    errorlist = []
    errorlist.append(testing_data['validate'].mean())

    for i in range(1,sections):
        
        testing_data = sample_proba(sample_data,i)
        testing_data['validate'] = np.where(testing_data['status_group']==testing_data['prediction'], 1, 0)
        current_Error = testing_data['validate'].mean()
        errorlist.append(current_Error)
#         print(current_Error)
    
    error = (sum(errorlist)/len(errorlist))
    
#     print('Mean error: ' + str(error))
#     print('Mean error: ' + str(error))
    
    return error

In [80]:
def x_fold_accuracy(sampled_data,sections,kernel_):
    
    errorlist = []

    
    for i in range(sections):
        section = i
        training_data = sampled_data[sampled_data['random_group'] != section]
        testing_data = sampled_data[sampled_data['random_group'] == section]
        model2 = svm.SVC(kernel=kernel_)
        model2.fit(training_data[training_data.columns.difference(['id','status_group','index'])].values, training_data['status_group'].values)
        testing_data['prediction'] = model2.predict(testing_data[testing_data.columns.difference(['id','status_group','index'])].values)
        testing_data['validate'] = np.where(testing_data['status_group']==testing_data['prediction'], 1, 0)
        current_Error = testing_data['validate'].mean()
        errorlist.append(current_Error)
        print(current_Error)

    
    average_accuracy = sum(errorlist)/len(errorlist)
    print('Average accuracy is: ' + str(average_accuracy))
    
    return average_accuracy

In [86]:
sampled_data = assign_samples(data_processed.sample(frac=1),20)

In [82]:
# %time run_x_fold(sampled_data,20)

In [87]:
run_x_fold(sampled_data,5)
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


0.7739568345323741

In [92]:
sampled_data

,id,status_group,amount_tsh_standard,gps_height_standard,longitude_standard,latitude_standard,population_standard,age_standard,permit_mapped,public_meeting_mapped,Arusha Rural,Arusha Urban,Babati,Bagamoyo,Bahi,Bariadi,Biharamulo,Bukoba Rural,Bukoba Urban,Bukombe,Bunda,Chamwino,Chato,Chunya,Dodoma Urban,Geita,Hai,Hanang,Handeni,Igunga,Ilala,Ileje,Ilemela,Iramba,Iringa Rural,Kahama,Karagwe,Karatu,Kasulu,Kibaha,Kibondo,Kigoma Rural,Kigoma Urban,Kilindi,Kilolo,Kilombero,Kilosa,Kilwa,Kinondoni,Kisarawe,Kishapu,Kiteto,Kondoa,Kongwa,Korogwe,Kwimba,Kyela,Lindi Rural,Lindi Urban,Liwale,Longido,Ludewa,Lushoto,Mafia,Magu,Makete,Manyoni,Masasi,Maswa,Mbarali,Mbeya Rural,Mbinga,Mbozi,Mbulu,Meatu,Meru,Misenyi,Missungwi,Mkinga,Mkuranga,Monduli,Morogoro Rural,Morogoro Urban,Moshi Rural,Moshi Urban,Mpanda,Mpwapwa,Mtwara Rural,Mtwara Urban,Mufindi,Muheza,Muleba,Musoma Rural,Mvomero,Mwanga,Nachingwea,Namtumbo,Nanyumbu,Newala,Ngara,Ngorongoro,Njombe,Nkasi,Nyamagana,Nzega,Pangani,Rombo,Rorya,Ruangwa,Rufiji,Rungwe,Same,Sengerema,Serengeti,Shinyanga Rural,Shinyanga Urban,Siha,Sikonge,Simanjiro,Singida Rural,Singida Urban,Songea Rural,Songea Urban,Sumbawanga Rural,Sumbawanga Urban,Tabora Urban,Tandahimba,Tanga,Tarime,Temeke,Tunduru,Ukerewe,Ulanga,Urambo,Uyui,afridev,cemo,climax,gravity,india mark ii,india mark iii,ksb,mono,nira/tanira,other,other - mkulima/shinyanga,other - play pump,other - rope pump,other - swn 81,submersible,swn 80,walimi,windmill,commercial,other,parastatal,unknown,user-group,colored,fluoride,good,milky,salty,unknown,annually,monthly,never pay,on failure,other,per bucket,unknown,dry,enough,insufficient,seasonal,unknown,dam,hand dtw,lake,machine dbh,other,rainwater harvesting,river,shallow well,spring,unknown,cattle trough,communal standpipe,communal standpipe multiple,dam,hand pump,improved spring,other,coloured,fluoride,fluoride abandoned,milky,salty,salty abandoned,soft,unknown,Internal,Lake Nyasa,Lake Rukwa,Lake Tanganyika,Lake Victoria,Pangani,Rufiji,Ruvuma / Southern Coast,Wami / Ruvu,Arusha,Dar es Salaam,Dodoma,Iringa,Kagera,Kigoma,Kilimanjaro,Lindi,Manyara,Mara,Mbeya,Morogoro,Mtwara,Mwanza,Pwani,Rukwa,Ruvuma,Shinyanga,Singida,Tabora,Tanga,Accra,Artisan,CEFA,Central government,Commu,DANIDA,DWE,DWSP,Danid,District council,GRUMETI,Government,ISF,KKKT _ Konde and DWE,Kilolo Star,LGA,Lawatefuka water sup,MUWSA,Not known,Olgilai village community,Private,Roman,TWE,UNICEF,WEDECO,WU,Water,Water Aid,World Vision,World vision,other,Action In A,African Development Bank,Biore,Cefa,Ces(gmbh),Danida,Dwsp,European Union,Government Of Tanzania,Grumeti,Hesawa,Isf,Isingiro Ho,Kirde,Lawatefuka Water Supply,Lottery Club,Mkinga Distric Coun,Not Known,Private,Private Individual,Roman,Rudep,Rwssp,Sobodo,Twe,Undp,Unicef,Water,Wateraid,World Vision,other,random_group
0,34310,functional,-0.191776,-0.543395,0.886320,0.639751,-0.055053,-1.117212,1.00000,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,54551,non functional,0.000000,0.000000,-0.969950,0.502321,0.000000,0.052669,1.00000,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0

In [26]:
sampled_data = assign_samples(data_processed.sample(frac=.1),10)

In [30]:
%time x_fold_accuracy(sampled_data,10,'poly')

os.system('say "your program has finished"')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


0.5589225589225589
0.5370370370370371
0.5572390572390572
0.5370370370370371
0.5050505050505051
0.5505050505050505
0.5505050505050505
0.5269360269360269
0.5505050505050505
0.5723905723905723
Average accuracy is: 0.5446127946127947
CPU times: user 1min 46s, sys: 820 ms, total: 1min 46s
Wall time: 1min 49s


0

In [ ]:
sampled_data

0